In [2]:
import pandas as pd
import time
import numpy as np
import re
import pickle
import string, unicodedata
from unidecode import unidecode

## Load Data

In [3]:
#data = pd.read_excel('data/data_komentar_clean_1.xlsx', sheet_name = 'Sheet1')
data = pd.read_excel('data/data_komentar_clean_1.xlsx', sheet_name = 'imbang')
komentar = data['komentar'].tolist()
label = data['label'].tolist()
kode = data['kode'].tolist()

## Proses CountVectorizer

In [4]:
from sklearn.feature_extraction.text import CountVectorizer      
countV = CountVectorizer() #membuat model CountVectorizer
##Membuat Matrix CountVectorizer
countV_matrix = countV.fit_transform(komentar) #melakukan fit data -> seluruh data komentar
print(countV_matrix.shape)
X = countV_matrix.toarray() #merubah tipe data ke array
#X= countV_matrix
Y = np.array(kode) #labet tiap (komentat [spam dan bukan spam])/ 1:0
y = Y

(3800, 13065)


## Proses Pemodelan

In [7]:
#Import Library
import numpy as np
from sklearn import svm
from sklearn.naive_bayes import ComplementNB

import pylab as pl
from sklearn.svm import SVC
#from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import StratifiedKFold
#from sklearn.model_selection import GridSearchCV

# Mencari parameter terbaik

## Proposes percobaan untuk masing masing parameter SVM

In [ ]:
K = 10 #pembagian data uji dan data latih
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=K) 
C_range = np.arange(1,2.1,0.1)
gamma_range = np.arange(1,2.1, 0.1)

C_range_ = []
gamma_range_ = []
rerata_akurasi_svm = []
count=0
for i in range(len(C_range)):
    for j in range(len(gamma_range)):
        C = C_range[i]
        gamma = gamma_range[j]
        akurasi_svm = []
        cSVM = svm.SVC(kernel='rbf', gamma=gamma, C=C) #model
        for train_index, test_index in kf.split(X):
            #membagi data latih dan data uji
            X_train, X_test = X[train_index], X[test_index] #pembagian data latih
            y_train, y_test = y[train_index], y[test_index]
    
            #Melatih Model 
            cSVM = cSVM.fit(X_train, y_train) #SVM
            
            #proses prediksi
            P_SVM = cSVM.predict(X_test)
            akurasi_svm.append(accuracy_score(y_test, P_SVM))
        print("iterasi ke-", count)
        C_range_.append(C)
        gamma_range_.append(gamma)
        
        rr = sum(akurasi_svm)/len(akurasi_svm)
        rerata_akurasi_svm.append(rr)
        count+=1
        #Menampilkan hasil 
        print("-"*30)
        print("C     : ", C)
        print("gamma : ", gamma)
        print("-"*20)
        print("Rata-rata Akurasi SVM: %0.2f" % (sum(akurasi_svm)/len(akurasi_svm)))
        print("="*30)
    
#proses mencari parameter dan akurasi terbaik
x = max(rerata_akurasi_svm)
#al = list(alpha_range)
index = rerata_akurasi_svm.index(x)
round(gamma_range_[index],2)
print("---------------------------")
print("gamma terbaik adalah :", round(gamma_range_[index],2))
print("    C terbaik adalah :", round(C_range_[index],2))
print("             Akurasi :", round(x, 2))

#MENYIMPAN HASIL
dict_komentar  = {"gamma":gamma_range_,"c":C_range_, "akurasi":rerata_akurasi_svm}
df = pd.DataFrame(dict_komentar,columns = ["gamma", 'c', "akurasi"])
writer = pd.ExcelWriter('data/parameter-akurasi-svm_CountVectorizer.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='svm-parameter-svm-countV')
#df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

iterasi ke- 0
------------------------------
C     :  1.0
gamma :  1.0
--------------------
Rata-rata Akurasi SVM: 0.60
iterasi ke- 1
------------------------------
C     :  1.0
gamma :  1.1
--------------------
Rata-rata Akurasi SVM: 0.59
iterasi ke- 2
------------------------------
C     :  1.0
gamma :  1.2000000000000002
--------------------
Rata-rata Akurasi SVM: 0.59
iterasi ke- 3
------------------------------
C     :  1.0
gamma :  1.3000000000000003
--------------------
Rata-rata Akurasi SVM: 0.59
iterasi ke- 4
------------------------------
C     :  1.0
gamma :  1.4000000000000004
--------------------
Rata-rata Akurasi SVM: 0.59
iterasi ke- 5
------------------------------
C     :  1.0
gamma :  1.5000000000000004
--------------------
Rata-rata Akurasi SVM: 0.58
iterasi ke- 6
------------------------------
C     :  1.0
gamma :  1.6000000000000005
--------------------
Rata-rata Akurasi SVM: 0.58
iterasi ke- 7
------------------------------
C     :  1.0
gamma :  1.7000000000000006

## Menyimpan hasil percobaan SVM

In [62]:
#MENYIMPAN HASIL
dict_komentar  = {"gamma":gamma_range_,"c":C_range_, "akurasi":rerata_akurasi_svm}
df = pd.DataFrame(dict_komentar,columns = ["gamma", 'c', "akurasi"])
writer = pd.ExcelWriter('data/parameter-akurasi-svm_CountVectorizer.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='svm-parameter-svm')
#df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

# Mencari Parameter Alpha

In [8]:
K = 10 #pembagian data uji dan data latih
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score#, precision_score

kf = KFold(n_splits=K)

alpha_range = np.arange(0.1,1.01,0.01)

rerata_akurasi_cnb = []

count=0
for i in range(len(alpha_range)): #melakukan interasi sebanyak data parameter C
    alpha = alpha_range[i]
    akurasi_cnb = []
    comNB = ComplementNB(alpha = alpha) #model
    tic = time.perf_counter()
    for train_index, test_index in kf.split(X):
            #membagi data latih dan data uji
        X_train, X_test = X[train_index], X[test_index] #pembagian data latih
        y_train, y_test = y[train_index], y[test_index]
    
        #Melatih Model 
        mCNB = comNB.fit(X_train, y_train) 
            
        #proses prediksi
        P_CNB = mCNB.predict(X_test)
        akurasi_cnb.append(accuracy_score(y_test, P_CNB))
    toc = time.perf_counter()
    print("iterasi ke-", count)    
    rr = sum(akurasi_cnb)/len(akurasi_cnb)
    rerata_akurasi_cnb.append(rr)
    count+=1
    #Menampilkan hasil akhir
    print("---------------------------------")
    print("alpha : ", alpha)
    print("----------------------------")
    print("Rata-rata Akurasi cnb: %0.2f" % (sum(akurasi_cnb)/len(akurasi_cnb)))
    print("---------------------------------", toc-tic, "second")
    
#proses mencari parameter dan akurasi terbaik
x = max(rerata_akurasi_cnb)
index = rerata_akurasi_cnb.index(x)
print("---------------------------")
print("alpha terbaik adalah :", round(alpha_range[index],2))
print("             Akurasi :", round(x, 2))

iterasi ke- 0
---------------------------------
alpha :  0.1
----------------------------
Rata-rata Akurasi cnb: 0.93
--------------------------------- 11.250250856929906 second
iterasi ke- 1
---------------------------------
alpha :  0.11
----------------------------
Rata-rata Akurasi cnb: 0.93
--------------------------------- 9.504741261816942 second
iterasi ke- 2
---------------------------------
alpha :  0.12
----------------------------
Rata-rata Akurasi cnb: 0.93
--------------------------------- 9.54688489683262 second
iterasi ke- 3
---------------------------------
alpha :  0.13
----------------------------
Rata-rata Akurasi cnb: 0.93
--------------------------------- 9.477715542726664 second
iterasi ke- 4
---------------------------------
alpha :  0.13999999999999999
----------------------------
Rata-rata Akurasi cnb: 0.93
--------------------------------- 9.649695087022991 second
iterasi ke- 5
---------------------------------
alpha :  0.14999999999999997
-------------------

iterasi ke- 43
---------------------------------
alpha :  0.5299999999999998
----------------------------
Rata-rata Akurasi cnb: 0.93
--------------------------------- 8.989106817646132 second
iterasi ke- 44
---------------------------------
alpha :  0.5399999999999998
----------------------------
Rata-rata Akurasi cnb: 0.93
--------------------------------- 9.361934205692137 second
iterasi ke- 45
---------------------------------
alpha :  0.5499999999999998
----------------------------
Rata-rata Akurasi cnb: 0.93
--------------------------------- 9.575620196023976 second
iterasi ke- 46
---------------------------------
alpha :  0.5599999999999997
----------------------------
Rata-rata Akurasi cnb: 0.93
--------------------------------- 9.555461438483462 second
iterasi ke- 47
---------------------------------
alpha :  0.5699999999999997
----------------------------
Rata-rata Akurasi cnb: 0.93
--------------------------------- 9.142056789795049 second
iterasi ke- 48
--------------------

iterasi ke- 86
---------------------------------
alpha :  0.9599999999999995
----------------------------
Rata-rata Akurasi cnb: 0.93
--------------------------------- 9.619735790370555 second
iterasi ke- 87
---------------------------------
alpha :  0.9699999999999995
----------------------------
Rata-rata Akurasi cnb: 0.93
--------------------------------- 9.842736407965731 second
iterasi ke- 88
---------------------------------
alpha :  0.9799999999999995
----------------------------
Rata-rata Akurasi cnb: 0.93
--------------------------------- 9.910605717431054 second
iterasi ke- 89
---------------------------------
alpha :  0.9899999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 9.726525124539648 second
iterasi ke- 90
---------------------------------
alpha :  0.9999999999999996
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 9.684693250039231 second
---------------------------
alpha t

## Menyimpan hasil percobaan CNB

In [9]:
round_akurasi_cnb = []
for i in rerata_akurasi_cnb:
    round_akurasi_cnb.append(round(i,2))
dict_komentar  = {"alpha":alpha_range,"akurasi":rerata_akurasi_cnb, "Akurasi round":round_akurasi_cnb}
df = pd.DataFrame(dict_komentar,columns = ["alpha","akurasi","Akurasi round"])
writer = pd.ExcelWriter('data/parameter-akurasi-cnb_CountVectorizer.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='cnb-parameter-cnb')

#menyimpan file
writer.save()